# Pruebas rápidas de la API Green Dream
Este notebook realiza comprobaciones básicas contra la API desplegada localmente:
- Verifica el endpoint `/api/health`
- Envía una petición de ejemplo a `/api/chat` y muestra la respuesta

Usa este notebook para validar que el servicio backend está disponible antes de integrar el chat en una web externa.

In [1]:
# Instalación mínima (si hace falta) e importaciones
import sys
try:
    import requests
except Exception:
    print("requests no está instalado. Instalando...")
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'requests'])
    import requests
from pprint import pprint

requests no está instalado. Instalando...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]4/5 [requests]


In [2]:
# Detectar la URL base adecuada (host.docker.internal cuando se ejecuta desde Docker host)
import requests
from requests.exceptions import RequestException

CANDIDATES = ['http://host.docker.internal:5001', 'http://localhost:5001']

def get_base_url(timeout=1):
    for url in CANDIDATES:
        try:
            r = requests.get(url + '/api/health', timeout=timeout)
            if r.status_code == 200:
                return url
        except RequestException:
            continue
    # Si ninguno responde, devolver el primero para diagnóstico
    return CANDIDATES[-1]

BASE_URL = get_base_url()
print('Usando BASE_URL =', BASE_URL)

Usando BASE_URL = http://localhost:5001


In [3]:
# Llamar a /api/health y mostrar resultado
try:
    r = requests.get(BASE_URL + '/api/health', timeout=5)
    r.raise_for_status()
    pprint(r.json())
except Exception as e:
    print('Error llamando a /api/health:', e)

{'assistant_initialized': True,
 'service': 'Green Dream Chat API',
 'status': 'healthy',
 'version': '1.0.0'}


In [5]:
# Ejemplo POST a /api/chat (puede fallar si no hay credenciales de Azure configuradas)
payload = {
    'message': 'Hola, ¿qué cursos recomiendas sobre desarrollo tienes?'
}
try:
    r = requests.post(BASE_URL + '/api/chat', json=payload, timeout=20)
    if r.status_code == 200:
        pprint(r.json())
    else:
        print(f'Respuesta inesperada {r.status_code}:', r.text)
except Exception as e:
    print('Error llamando a /api/chat:', e)

{'response': '¡Hola! 🌟 Qué genial que quieras aprender sobre desarrollo '
             'sostenible, ¡es un gran paso hacia un futuro más verde! 🌍 Aquí '
             'en Green Dream tenemos un curso perfecto para ti si estás '
             'empezando o quieres entender los fundamentos del desarrollo '
             'sostenible.\n'
             '\n'
             '### **Curso recomendado: Fundamentos de Sostenibilidad para '
             'Jóvenes**\n'
             '📚 Este curso es una **introducción ideal** al mundo del '
             'desarrollo sostenible, especialmente diseñado para jóvenes como '
             'tú. Aprenderás sobre los conceptos fundamentales del desarrollo '
             'sostenible, los Objetivos de Desarrollo Sostenible (ODS) de la '
             'ONU y cómo puedes aplicarlos en tu vida diaria para generar un '
             'impacto positivo.  \n'
             '\n'
             '💻 **Modalidad:** 100% online, así que puedes aprender desde la '
             'comodidad

## Notas
- Si `/api/chat` devuelve error 500 es probable que falten las credenciales de Azure (AZURE_AI_ENDPOINT / AZURE_AI_KEY).
- Copia `config/.env.example` a `config/.env` y rellena las variables antes de ejecutar en tu máquina: 

```powershell
Copy-Item .\config\.env.example .\config\.env
notepad .\config\.env
# Rellenar AZURE_AI_ENDPOINT y AZURE_AI_KEY
```

- En entornos de producción usa mecanismos de secretos gestionados (Azure KeyVault, Docker secrets, etc.).